In [6]:
import numpy as np

In [7]:
datos = np.array([["audi", "gas", "std", 1],
                 ["audi", "gas", "std", 1],
                 ["audi", "gas", "std", 0],
                 ["audi", "gas", "std", 2],
                 ["mazda", "gas", "std", 1],
                 ["mazda", "gas", "std", 1],
                 ["mazda", "gas", "std", 2],
                 ["peugot", "diesel", "rwd", 2],
                 ["peugot", "diesel", "turbo", 2],
                 ["volvo", "gas", "turbo", 0],
                 ["volvo", "gas", "std", 1],
                 ["volvo", "gas", "turbo", 0]])

new_data = np.array([[99.8, 176.6, 66.2, 54.3],
                     [99.4, 176.6, 66.4, 54.3],
                     [99.8, 177.3, 66.3, 53.1],
                     [105.8, 192.7, 71.4, 55.7],
                     [95.3, 169, 65.7, 49.6],
                     [98.8, 177.8, 66.5, 53.7],
                     [98.8, 177.8, 66.5, 55.5],
                     [107.9, 186.7, 68.4, 56.7],
                     [107.9, 186.7, 68.4, 56.7],
                     [104.3, 188.8, 67.2, 57.5],
                     [109.1, 188.8, 68.9, 55.5],
                     [109.1, 188.8, 68.8, 55.5]])

fisher_length = 0.3336
fisher_width = 0.2892
fisher_height = 0.4047
fisher_highway_mpg = 0.1335

In [18]:
#Calcular correlación
#Pij = 
print(new_data[:,0])

[ 99.8  99.4  99.8 105.8  95.3  98.8  98.8 107.9 107.9 104.3 109.1 109.1]


In [8]:
def return_unique_characteristics(column: int) -> list:
    characteristics = datos[:,column].tolist()
    characteristics

    unique_characteristics = []

    for characteristic in characteristics:
        if not(characteristic in unique_characteristics):
            unique_characteristics.append(characteristic)
    return unique_characteristics

def create_dictionary(unique_characteristics: list, total_class: int) -> dict:
    dictionary = dict()

    for characteristic in unique_characteristics:    
        for i in np.arange(total_class):
            key = characteristic + str(i)
            dictionary.setdefault(key, 0)
    return dictionary

In [9]:
total_class = 3
unique_characteristics = return_unique_characteristics(column=0)
dictionary = create_dictionary(unique_characteristics=unique_characteristics, total_class=total_class)
total_sum = 0
for characteristic in unique_characteristics:
    simple_sum = 0
    ni = 0
    for i in np.arange(total_class):
        key = characteristic + str(i)
        simple_sum += dictionary[key] / datos.shape[0]
        ni += dictionary[key]
    print(simple_sum)
    simple_sum = 1 - simple_sum
    total_sum += (simple_sum * ni) / datos.shape[0] 
print(total_sum)

0.0
0.0
0.0
0.0
0.0
